<a href="https://colab.research.google.com/github/HwangHanJae/Recommentaion_Study/blob/main/inflearn_recsys/user_group_rec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 사용자 집단별로 추천하기

# 데이터 읽기

무비렌즈의 유저의 정보(u.user) 읽기

In [18]:
import os
import pandas as pd
#베이스 경로 설정
base = '/content/drive/MyDrive/RecoSys/Data'

# u.user 파일 경로 설정
u_user_path = os.path.join(base, 'u.user')

#필요한 컬럼 정의
u_cols = ['user_id','age','sex','occupation','zip_code']

#데이터 읽어오기
users = pd.read_csv(u_user_path, sep='|', names = u_cols, encoding='latin-1')
#users 데이터 프레임에 인덱스(user_id) 지정
#users = users.set_index('user_id')

#상위 5개
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


무비렌즈의 영화의 정보(u.item) 읽기

In [19]:
#u.item의 파일 경로 설정
u_item_path = os.path.join(base, 'u.item')

#필요한 컬럼 정의
i_cols = ['movie_id','title','release date','video release date','IMDB URL','unknown','Action',
          'Adventure','Animation','Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama',
          'Fantasy','Film-Noir','Horror','Musical', 'Mystery','Romance','Sci-Fi','Thriller','War','Western']

# 데이터 읽어오기
movies = pd.read_csv(u_item_path, sep='|',names =i_cols, encoding='latin-1')

# movie_id와 title만 가져오기
movies = movies[['movie_id','title']]

#상위 5개
movies.head()

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


무비렌즈의 평점 정보(u.data)읽기

In [20]:
#u.data의 파일경로 지정
u_data_path = os.path.join(base, 'u.data')

#필요한 컬럼 정의
r_cols = ['user_id', 'movie_id','rating','timestamp']

#데이터 읽어오기
ratings = pd.read_csv(u_data_path, sep='\t',names = r_cols, encoding='latin-1')

#ratings 데이터 프레임에 인덱스(user_id) 지정
#ratings = ratings.set_index('user_id')

#timestamp 제거
ratings = ratings.drop('timestamp',axis=1)

#상위 5개
ratings.head()

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


# 훈련/테스트 세트 분리

In [21]:
# zip()함수 예시
x = [1,2,3,4]
y = ['one','two','three','four']

for x, y in zip(x, y):
  print(x, y)

1 one
2 two
3 three
4 four


In [22]:
from sklearn.model_selection import train_test_split
import numpy as np

#원본을 보존하기 위하여 copy
x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, stratify=y)

#정확도(RMSE)를 계산하는 함수
#best-seller 방식으로 구한 예측값의 RMSE를 계산하는 코드
def RMSE(y_true, y_pred):
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)
  return np.sqrt(np.mean((y_true - y_pred) ** 2))

# 모델별 RMSE를 계산하는 함수
def score(model):
  # user_id와 movie_id를 짝을 지어 id_pairs에 할당
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  # id_paris에 user와 movie를 하나씨 꺼내면서 model방식으로 y_pred을 예측
  y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
  # 실제 rating 값
  y_true = np.array(x_test['rating'])
  return RMSE(y_true, y_pred)

#훈련을 할 x_train
train_mean = x_train.groupby(['movie_id'])['rating'].mean()
def best_seller(user_id, movie_id):
  #train 세트에 movie_id가 존재하면 rating값을 얻어지만
  #존재하지 않다면 평균(3.0)값을 줌
  try:
    
    rating = train_mean[movie_id]
  except:
    rating = 3.0
  return rating

score(best_seller)

1.0270137380451878

# 성별에 따른 예측값 계산

In [23]:
#users 데이터프레임과 x_train 데이터프레임을 merge
merged_ratings = pd.merge(x_train, users)

users = users.set_index('user_id')

# 성별에 따라서 영화에 대해 평균 평점을 내림
g_mean = merged_ratings[['movie_id','sex','rating']].groupby(['movie_id','sex'])['rating'].mean()

In [26]:
g_mean

movie_id  sex
1         F      3.857143
          M      3.939271
2         F      3.312500
          M      3.235294
3         F      2.700000
                   ...   
1675      M      3.000000
1676      M      2.000000
1677      F      3.000000
1680      M      2.000000
1682      M      3.000000
Name: rating, Length: 3047, dtype: float64

In [25]:
#pivot 테이블을 이용하여 sparse matrix 형태로 만들어줌
#유저가 영화에 대해 어떤 평점을 내렸는지 알 수 있음
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')
rating_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1669,1671,1672,1673,1674,1675,1676,1677,1680,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,NaN,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# 성별 기준 추천
def cf_gender(user_id, movie_id):
  # 입력받은 movie_id가 rating_matrix에 존재한다면
  if movie_id in rating_matrix.columns:
    # 해당 유저의 성별을 확인
    gender = users.loc[user_id]['sex']
    # 성별이 g_mean에 존재하는지 확인
    if gender in g_mean[movie_id].index:
      gender_rating = g_mean[movie_id][gender]
    # 해당영화에 대하여 남성 혹은 여성만 평가를 했을 수도 있음
    else:
      gender_rating = 3.0
  # movie_id가 rating_matrix에 존재하지 않을수도 있음
  else:
    gender_rating = 3.0
  return gender_rating

score(cf_gender)



1.0353697157159687

베스트셀러 방식으로 추천을 적용하면 RMSE가 1점 정도 차이가 나는것으로 볼 수 있다.

1점이라는 것이 값 자체는 작지만 3점(보통)기준으로 2점(부정), 4점(긍정)처럼 이 영화에 대하여 긍정적인지 부정적인지 판단하기에는 적절하지 않을 수 있다.
